# Data Acquisition Example: Testing WLAN Client Receiver System Noise

Test data for WLAN is implemented with the the iperf support in [ssmdevices](https://gitlab.nist.gov/ssm/ssmdevices). It produces many more columns of data than we need, so we just use the two defined in `ipc_columns`: throughput and a timestamp.

The results of the tests and the corresponding test conditions are stored in a [flat database](https://en.wikipedia.org/wiki/Flat_file_database) stored in SQLite format. It is implemented as a typical intended use case of `labbench.RelationalDataInSQLite`. The test conditions are implemented as states in the attenuators, `iperf`, and `wlan`, so all we need to log these results into the database is to add the `db.on_set` call. Any states that are changed after that function call (for example, with the for loops in the acquisition code) become columns in the database; these values are automatically kept up to date and written to the database on calls to `db.write`.

In [ ]:
# This should be run in python 3.6.x from the computer connected to the AP side

%pylab inline
import time,os
import ssmdevices as ssm
import labbench as lb
import pandas as pd

def meshpoints (x1, x2, *args):
    ''' For input of N-dimensions of grid point vectors (x1, x2, ..., xN),
        where each (x1, x2, ... xN) has dimension (M1, M2, ..., MN),
        returns an array with shape (M1*M2*...*MN,N) that consists of the
        unique points in the multi-dimensional grid with sampled at
        the vector points (x1, x2, ..., xN) in each axis. This is also known
        as the the cartesian product of (x1 ... xN).
    '''
    ret = np.array(np.meshgrid(*((x1,x2)+args)))
    return ret.reshape([ret.shape[0],np.prod(ret.shape[1:])]).T

setup_time = 5 # seconds
acquisition_time = 1 #  seconds

# iperf_columns       = ['iperf_bits_per_second','iperf_timestamp']
sweep_points       = meshpoints(list(range(55,76,20)),[110]+list(range(0,21,20)))

lb.show_messages('info')

class Testbed(lb.Testbed):       
    def make(self):
        ''' lb.Testbed calls this automatically when we call Testbed(config)
        '''
        self.c = ssm.instruments.MiniCircuitsRCDAT('11604210008')
        self.e0 = ssm.instruments.MiniCircuitsRCDAT('11604210014')
        self.iperf_client = ssm.software.IPerf('10.0.0.2', bind='10.0.0.3', port=5010, interval=0.1)
        self.iperf_server = ssm.software.IPerf(bind='10.0.0.2', port=5010)
        self.wlan = ssm.software.WLANStatus('Wi-Fi', ssid='EnGenius1')
        self.db = lb.StatesToSQLite(time.strftime("%Y-%m-%d-%H%M%S with shuffle and absorber"))

        # Log all state changes in lte_laa and each client in iperf_clients
        self.db.observe_states([self.c,self.e0,self.iperf_server,self.iperf_client,self.wlan])
        
    def acquire(self, duration):
        ''' acquire for `duration` seconds and return a dictionary of
            client and server data frames
        '''
        # Cycle through port in iperf server & client to avoid some strange
        # errors involving stale ports in windows
        p = self.iperf_server.settings.port
        self.iperf_client.settings.port = self.iperf_server.settings.port = p+1            
        
        self.iperf_server.start()
        self.iperf_client.start()

        lb.logger.info('acquiring for {}s'.format(duration))
        lb.sleep(setup_time)

        client = self.iperf_client.fetch()
        server = self.iperf_server.fetch()

        lb.logger.debug('  iperf_client and server returned {} and {} rows'\
                       .format(len(client),len(server)))

        self.iperf_server.kill()
        self.iperf_client.kill()
        
        return {'client': client,
                'server': server}

    def startup(self):
        lb.panel(self, ncols=5) # Heads-up display for the notebook
        
        lb.logger.info('setup start')
        self.t0 = time.time()
        
        self.wlan.interface_disconnect()
        
        self.c.state.attenuation = 50
        self.e0.state.attenuation = 110
        
        # Throwaway acquisition to get into a favorable state
        self.acquire(setup_time)    
        lb.logger.info('setup done')
    
    def cleanup(self):
        # End any stale iperf processes in the correct order
        self.iperf_server.kill()
        self.iperf_client.kill()
        
        # Leave attenuators in a sensible state
        self.c.state.attenuation = 50
        self.e0.state.attenuation = 110
        
        # Load the sqlite database and save a copy into a csv
        df = lb.read(os.path.join(self.db.path, 'master.db'))
        df.to_csv(os.path.join(self.db.path,'master.csv'))        
        
        lb.logger.info('finished with testbed after {}s'\
                       .format(time.time()-self.t0))

for i in range(1):
    states = {}

    np.random.shuffle(sweep_points) # randomize?    
    
    # This `with' block ensures sockets and iperf subprocesses are
    # closed correctly when the script ends, even if exceptions are raised
    with Testbed() as testbed:
        for testbed.c.state.attenuation, testbed.e0.state.attenuation\
            in lb.log_progress(sweep_points):

            try:
                # Fresh connection in the new attenuation state
                testbed.wlan.interface_reconnect(5)
            except TimeoutError as e:
                # Blank data
                lb.logger.debug(str(e))
                lb.logger.debug('No connection to AP - skipping data acquisition')
                row = {}           
            else:
                # Take data
                row = testbed.acquire(acquisition_time)
            finally:
                # Update all WLAN states before append 
                testbed.wlan.refresh()
                testbed.db.append(row)
                testbed.db.write()

In [ ]:
np.random.shuffle(sweep_points)

In [ ]:
df = lb.read(r'C:/Users/dkuester/Documents/src/ssmdevices/examples/2018-10-30-141739 with shuffle and absorber/master.db')
df2 = lb.read(r'C:\Users\dkuester\Documents\src\ssmdevices\examples\2018-10-30-154819 with shuffle and absorber\master.db')

df=df.sort_values(['c_attenuation','e0_attenuation'])
df2=df2.sort_values(['c_attenuation','e0_attenuation'])
df=df.set_index(['c_attenuation','e0_attenuation'])
df2=df2.set_index(['c_attenuation','e0_attenuation'])

# i = (df['wlan_state']=='connected')&(df2['wlan_state']=='connected')

(df['wlan_signal']-df2['wlan_signal']).hist(bins=20)
print((df[i]['wlan_signal']-df2[i]['wlan_signal']).mad())

In [ ]:
df.columns

In [ ]:
import labbench as lb
import importlib
lb = importlib.reload(lb)

df = lb.read_relational(r'C:/Users/dkuester/Documents/src/ssmdevices/examples/2018-10-30-141739 with shuffle and absorber/master.db',                        
                        'iperf_client', prepend_column_name=False,
                        target_cols=['iperf_destination_address','iperf_destination_port', 'iperf_bits_per_second', 'iperf_timestamp'],)

df.columns

In [ ]:
df.shape

In [ ]:
lb.read(r'C:\Users\dkuester\Documents\src\ssmdevices\examples\2018-10-30-154819 with shuffle and absorber\0 2018-10-30 154837.796099\iperf_client.csv')

In [ ]:
df2

In [ ]:
df2.sort_values(['c_attenuation','e0_attenuation'])